# Applying GNN Models

In this lecture, we will continue using the Cora example from the previous lesson. You will learn about:

- Unsupervised GRL
- GNNs for Supervised Downstream Tasks

We will also compare these methods with the approach discussed in the previous lesson.

In [26]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html

import os.path as osp
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

import torch.nn as nn
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
from torch_geometric.loader import LinkNeighborLoader

2.1.1


## Unsupervised Graph Representation Learning with GraphSAGE
Since we aim to learn graph representations through an unsupervised method, we do not use node labels for training.<br>

We assume that if there is a link between a pair of nodes, those nodes should have similar embeddings. Conversely, if there is no link between a pair of nodes, their embeddings should be dissimilar.

Based on this assumption, we can define the following loss function:
 
\begin{equation}
\text{Loss} = - \left( \log \left( \sigma(h_u^{\top} h_v) \right) - \sum_{i=1}^k \log \left( \sigma(h_u^{\top} h_{n_i}) \right) \right) , n_i \sim P_V
\end{equation}

- $\log \left( \sigma (h_u^{\top} h_v) \right)$:  The similarity between the positive sample pair (i.e., true neighbors). Maximizing this term means you want the similarity of positive samples to be as high as possible.
  
- $- \sum_{i=1}^k \log \left( \sigma (h_u^{\top} h_{n_i}) \right)$: The similarity between the negative sample pairs (i.e., non-neighbors). Minimizing this term means you want the similarity of negative samples to be as low as possible.

Once the embeddings are obtained, they are fed into an additional classifier for node classification.

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(root=path, name='Cora', transform=NormalizeFeatures())
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [28]:
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv(in_channels, hidden_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        return x

In [29]:
# define neighbor sampler
train_loader = LinkNeighborLoader(
    data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)

In [30]:
for batch in train_loader:
    print(batch)
    break

Data(x=[2448, 1433], edge_index=[2, 7895], y=[2448], train_mask=[2448], val_mask=[2448], test_mask=[2448], n_id=[2448], e_id=[7895], input_id=[256], edge_label_index=[2, 512], edge_label=[512])


In [31]:
print("Edge label index: containing both positive and negative edges")
print(batch.edge_label_index)

print("Edge label: 1 stands for positive and 0 stands for negative node pair(edge)")
print(batch.edge_label)

Edge label index: containing both positive and negative edges
tensor([[308, 353, 703,  ..., 162, 654, 258],
        [686, 688, 704,  ..., 278, 464, 675]])
Edge label: 1 stands for positive and 0 stands for negative node pair(edge)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(data.num_node_features, hidden_channels=64, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
model = model.to(device)
x, edge_index = data.x.to(device), data.edge_index.to(device)

In [33]:
# define training and testing functions
def train():
    model.train()

    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        embedding = model(batch.x, batch.edge_index)
        embedding_src = embedding[batch.edge_label_index[0]]
        embedding_dst = embedding[batch.edge_label_index[1]]
        pred = (embedding_src * embedding_dst).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pred.size(0)

    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    out = model(data.x.to(device), data.edge_index.to(device)).cpu() 

    clf = LogisticRegression()
    clf.fit(out[data.train_mask], data.y[data.train_mask])

    val_acc = clf.score(out[data.val_mask], data.y[data.val_mask])
    test_acc = clf.score(out[data.test_mask], data.y[data.test_mask])

    return val_acc, test_acc

In [34]:
for epoch in range(1, 101):
    loss = train()
    val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Loss: 5.1395, Val: 0.3280, Test: 0.3180
Epoch: 002, Loss: 4.6441, Val: 0.3380, Test: 0.3070
Epoch: 003, Loss: 4.5985, Val: 0.4560, Test: 0.4170
Epoch: 004, Loss: 4.4559, Val: 0.5420, Test: 0.5190
Epoch: 005, Loss: 4.2841, Val: 0.6020, Test: 0.6090
Epoch: 006, Loss: 4.1689, Val: 0.6200, Test: 0.6120
Epoch: 007, Loss: 4.0831, Val: 0.6400, Test: 0.6580
Epoch: 008, Loss: 3.9265, Val: 0.6580, Test: 0.6820
Epoch: 009, Loss: 3.9477, Val: 0.6740, Test: 0.6820
Epoch: 010, Loss: 3.8382, Val: 0.6680, Test: 0.6950
Epoch: 011, Loss: 3.8586, Val: 0.6780, Test: 0.6930
Epoch: 012, Loss: 3.7633, Val: 0.6980, Test: 0.7190
Epoch: 013, Loss: 3.7493, Val: 0.6880, Test: 0.7070
Epoch: 014, Loss: 3.7360, Val: 0.7060, Test: 0.7170
Epoch: 015, Loss: 3.7332, Val: 0.7080, Test: 0.7140
Epoch: 016, Loss: 3.6879, Val: 0.7020, Test: 0.7130
Epoch: 017, Loss: 3.7057, Val: 0.7080, Test: 0.7380
Epoch: 018, Loss: 3.6804, Val: 0.7220, Test: 0.7430
Epoch: 019, Loss: 3.6435, Val: 0.7180, Test: 0.7580
Epoch: 020, 

## Performance comparison
Recall that in the previous examples, we performed node classification in 3 different ways.
1. Bag of word + MLP with `Accuracy:0.6`
2. Node2vec + Logistic regression with `Accuracy:0.703`
3. Node2vec with Bag of word + Logistic regression with `Accuracy:0.707`
4. GraphSAGE: with Bag of word + Logistic regression with `Accuracy:0.803`

We make use of node feature and graph structure at the same time and boost the accuracy up to **0.791** with a simple two-layer `GraphSAGE`.

## End-to-end semi-supervised learning with Graph Convolution Network(GCN)
Previously, we adopt a two stage classification pipeline where we first extract network feature via unsupervised learning then utilize a classifier to predict its label. <br>
The two-stage design could be suboptimal since the network features were not extracted for specific task. <br>
Therefore, we now employ an end-to-end approach. This ensures that the features learned by the model are directly aligned with the specific task, potentially leading to better performance.

In [35]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(hidden_channels, out_channels, cached=True,
                             normalize=True)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [39]:
dim = 64
model = GCN(dataset.num_features, dim, dataset.num_classes)
model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-4)
print(model)

GCN(
  (conv1): GCNConv(1433, 64)
  (conv2): GCNConv(64, 7)
)


In [40]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_weight)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_weight).argmax(dim=-1)
    mask = data.test_mask
    accs = (int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

In [41]:
for epoch in range(200):
    loss = train()
    test_acc = test()
    print(f"Loss:{loss:.4f} Testing accuracy:{test_acc:.4f}")

Loss:1.9458 Testing accuracy:0.3040
Loss:1.9438 Testing accuracy:0.3250
Loss:1.9422 Testing accuracy:0.3630
Loss:1.9404 Testing accuracy:0.4290
Loss:1.9387 Testing accuracy:0.4990
Loss:1.9373 Testing accuracy:0.5590
Loss:1.9349 Testing accuracy:0.6140
Loss:1.9332 Testing accuracy:0.6590
Loss:1.9308 Testing accuracy:0.6930
Loss:1.9286 Testing accuracy:0.7270
Loss:1.9260 Testing accuracy:0.7470
Loss:1.9242 Testing accuracy:0.7510
Loss:1.9213 Testing accuracy:0.7640
Loss:1.9187 Testing accuracy:0.7720
Loss:1.9171 Testing accuracy:0.7820
Loss:1.9125 Testing accuracy:0.7870
Loss:1.9120 Testing accuracy:0.7930
Loss:1.9083 Testing accuracy:0.7940
Loss:1.9039 Testing accuracy:0.7970
Loss:1.9019 Testing accuracy:0.7960
Loss:1.9004 Testing accuracy:0.7970
Loss:1.8966 Testing accuracy:0.7970
Loss:1.8927 Testing accuracy:0.7960
Loss:1.8894 Testing accuracy:0.7970
Loss:1.8869 Testing accuracy:0.7980
Loss:1.8824 Testing accuracy:0.7980
Loss:1.8810 Testing accuracy:0.7990
Loss:1.8751 Testing accuracy

## Performance comparison
Recall that in the previous examples, we performed node classification in 3 different ways.
1. Bag of word + MLP with `Accuracy:0.6`
2. Node2vec + Logistic regression with `Accuracy:0.703`
3. Node2vec with Bag of word + Logistic regression with `Accuracy:0.707`
4. GraphSAGE: with Bag of word + Logistic regression with `Accuracy:0.803`
4. GCN: end-to-end learning with `Accuracy:0.818`

From this example, we clearly figure out that using GCN with end2end training leads to the best performance since the feature extracted could be learned to optimize the node classification task.

## Applying different GNN backbone layer
The full list of implemented GNN could be found in [here.](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers)

In [15]:
from torch_geometric.nn import GCNConv,GATConv, GraphSAGE
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, gnn_type):
        super().__init__()
        if gnn_type == "GCN":
            self.GNN = GCNConv
        elif gnn_type == "SAGE":
            self.GNN = GraphSAGE
        elif gnn_type == "GAT":
            self.GNN = GATConv
        
        self.conv1 = self.GNN(in_channels, hidden_channels)
        self.conv2 = self.GNN(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [16]:
dim = 32
gnn_type = "GAT"
model = GNN(dataset.num_features, dim, dataset.num_classes,gnn_type=gnn_type)
model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters())
print(model)

GNN(
  (conv1): GATConv(1433, 32, heads=1)
  (conv2): GATConv(32, 7, heads=1)
)


# HW: Link prediction with GNN
1. Try different GNN layer
2. Try to optimize the performance by stacking multiple layers
3. Report the best accuracy on testing set and the best model configuration(e.g., how many layers?)

In [17]:
# Let's practice how to use GNN for link prediction
# First we need to load the Cora dataset

from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import negative_sampling


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=True),
])
dataset = Planetoid(path, name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]

In [18]:
print("--------Training data------")
print(train_data)
print("Training edges:")
print(train_data.edge_label_index)
print("Labels")
print(train_data.edge_label)

print()
print("--------Testing data------")
print(test_data)
print("Testing edges:")
print(test_data.edge_label_index)
print("Labels")
print(test_data.edge_label)

--------Training data------
Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[8976], edge_label_index=[2, 8976])
Training edges:
tensor([[1282,  353, 1986,  ..., 2699, 1780, 2192],
        [1483, 1173, 2009,  ...,  506, 1334,  914]])
Labels
tensor([1., 1., 1.,  ..., 0., 0., 0.])

--------Testing data------
Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054])
Testing edges:
tensor([[ 793,  225,  103,  ..., 2237, 1341, 2178],
        [1896, 2255,  484,  ...,  685,  192, 1389]])
Labels
tensor([1., 1., 1.,  ..., 0., 0., 0.])


In [19]:
class MyGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        ############################################################################
        # TODO: Your code here! 
        # create you GNN layer here. 
        # try to use different GNN backbone layer or stacking multiple layer to boost performance
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
        ############################################################################

    def forward(self, x, edge_index):
        ############################################################################
        # TODO: Your code here! 
        # Apply the forward pass according to your GNN layers
        # you shoud return the embedding of each node (x has shape [num_nodes, dim])    
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)  
        ############################################################################
        return x
    
    def get_prediction(self, node_embedding, edges):
        # In this function, we have the node embedding and edges as input
        # Input shapes:
        #      node_embedding: (|V|, out_channels)
        #      edges: (2, number of edges)
        # To generate such output, we use the inner product of embeddings of two nodes
        # The output is to generate a scalar for each pair of edge
        embedding_first_node = node_embedding[edges[0]]
        embedding_second_node = node_embedding[edges[1]]
        ############################################################################
        # TODO: Your code here! 
        # implement the element-wise product as edge feature for link prediction
        inner_product = torch.sum(embedding_first_node * embedding_second_node, dim=-1)
        
        ############################################################################
        return inner_product

In [20]:
############################################################################
# TODO: Your code here! 
# initiate your GNN model and select the criterion for link prediction

model = MyGNN(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
############################################################################

In [21]:
# Implement the train function
def train():
    model.train()
    optimizer.zero_grad()
    embedding = model(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    
    # Please assign the target for negative edges
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)
    
    # make prediction
    prediction = model.get_prediction(embedding, edge_label_index).view(-1)
    
    # optimization
    loss = criterion(prediction, edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [22]:
# Implement the test function
@torch.no_grad()
def test(data):
    model.eval()
    embedding = model(data.x, data.edge_index)
    
    # use the sigmoid function to normalize our prediction into [0,1]
    out = model.get_prediction(embedding, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [23]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')


Epoch: 001, Loss: 0.6932, Val: 0.6935, Test: 0.7040
Epoch: 002, Loss: 0.7008, Val: 0.7408, Test: 0.7591
Epoch: 003, Loss: 0.6936, Val: 0.6889, Test: 0.6780
Epoch: 004, Loss: 0.6934, Val: 0.5864, Test: 0.5652
Epoch: 005, Loss: 0.6940, Val: 0.6402, Test: 0.5956
Epoch: 006, Loss: 0.6944, Val: 0.6259, Test: 0.6506
Epoch: 007, Loss: 0.6943, Val: 0.6932, Test: 0.6671
Epoch: 008, Loss: 0.6939, Val: 0.6657, Test: 0.6748
Epoch: 009, Loss: 0.6935, Val: 0.6579, Test: 0.6609
Epoch: 010, Loss: 0.6933, Val: 0.6424, Test: 0.6254
Epoch: 011, Loss: 0.6933, Val: 0.6070, Test: 0.5935
Epoch: 012, Loss: 0.6933, Val: 0.4723, Test: 0.5012
Epoch: 013, Loss: 0.6934, Val: 0.4615, Test: 0.4950
Epoch: 014, Loss: 0.6934, Val: 0.4647, Test: 0.4978
Epoch: 015, Loss: 0.6933, Val: 0.5046, Test: 0.5264
Epoch: 016, Loss: 0.6933, Val: 0.6678, Test: 0.6663
Epoch: 017, Loss: 0.6931, Val: 0.7031, Test: 0.7098
Epoch: 018, Loss: 0.6930, Val: 0.7287, Test: 0.7264
Epoch: 019, Loss: 0.6928, Val: 0.7227, Test: 0.7138
Epoch: 020, 